# **D. Pricing and Ratings**

<span style="font-size: 14px;"><b>1. If a Meat Lovers pizza costs $12 and Vegetarian costs $10 and there were no charges for changes&nbsp;</b></span> **\- how much money has Pizza Runner made so far if there are no delivery fees?**

In [1]:
with cte as(SELECT 
*,
CASE
	WHEN pn.pizza_name='Meatlovers' Then 12
	ELSE 10
END pizza_cost
FROM new_customer_orders c
JOIN pizza_names pn
	ON c.pizza_id = pn.pizza_id
	ORDER BY c.order_id)
	SELECT
		SUM(pizza_cost) as total_revenue
	FROM cte

(1 row(s) affected)

Total execution time: 00:00:00.009

total_revenue
160


<span style="font-size: 14px;"><b>2. What if there was an additional $1 charge for any pizza extras? Add cheese is $1 extra</b></span>

**3. The Pizza Runner team now wants to add an additional ratings system that allows customers to rate their runner, how would you design an additional table for this new dataset - generate a schema for this new table and insert your own data for ratings for each successful customer order between 1 to 5.**

In [4]:
CREATE TABLE ratings(
   customer_id    INT     NOT NULL,
   order_id       INT     NOT NULL,
   runner_id      INT     NOT NULL,
   rating         INT
);

Commands completed successfully

Total execution time: 00:00:00.006

In [5]:
INSERT INTO 
ratings (customer_id,order_id,runner_id,rating) 
VALUES  (101, 1, 1, 3),
		(101, 2, 1, 5),
		(102, 3, 1, 4),
		(103, 4, 2, 4),
		(104, 5, 3, 5),
		(101, 6, 3, null ),
		(105, 7, 2, 5),
		(102, 8, 2, 3),
		(103, 9, 2, null ),
		(104, 10, 1, 4);

Commands completed successfully

Total execution time: 00:00:00.001

In [6]:
SELECT 
*
FROM ratings;

(10 row(s) affected)

Total execution time: 00:00:00.006

customer_id,order_id,runner_id,rating
101,1,1,3
101,2,1,5
102,3,1,4
103,4,2,4
104,5,3,5
101,6,3,NULL
105,7,2,5
102,8,2,3
103,9,2,NULL
104,10,1,4


<span style="font-size: 14px;"><b>4. Using your newly generated table - can you join all of the information together to form a table which has the following information for successful deliveries?</b></span>

- **customer\_id**
- **order\_id**
- **runner\_id**
- **rating**
- **order\_time**
- **pickup\_time**
- **Time between order and pickup**
- **Delivery duration**
- **Average speed**
- **Total number of pizzas**

In [7]:
SELECT 
	c.customer_id,
	c.order_id,
	r.runner_id,
	ra.rating,
	c.order_time,
	r.pickup_time,
	((DATE_PART('day', r.pickup_time::timestamp - c.order_time::timestamp) * 24 + 
               DATE_PART('hour', r.pickup_time::timestamp - c.order_time::timestamp)) * 60 +
               DATE_PART('minute', r.pickup_time::timestamp - c.order_time::timestamp)) as Time_between_order_and_pickup,
	r.duration,
	ROUND(AVG((r.distance/r.duration)*100),2) as Average_speed,
	COUNT(c.pizza_id) as Total_number_of_pizzas
FROM new_customer_orders c
JOIN new_runner_orders r
	ON c.order_id = r.order_id
JOIN ratings ra
	ON c.order_id = ra.order_id
	WHERE r.cancellation is null
	GROUP BY
		c.customer_id,
		c.order_id,
		r.runner_id,
		ra.rating,
		c.order_time,
		r.pickup_time,
		((DATE_PART('day', r.pickup_time::timestamp - c.order_time::timestamp) * 24 + 
               DATE_PART('hour', r.pickup_time::timestamp - c.order_time::timestamp)) * 60 +
               DATE_PART('minute', r.pickup_time::timestamp - c.order_time::timestamp)),
		r.duration
	ORDER BY customer_id

(8 row(s) affected)

Total execution time: 00:00:00.009

customer_id,order_id,runner_id,rating,order_time,pickup_time,time_between_order_and_pickup,duration,average_speed,total_number_of_pizzas
101,1,1,3,2020-01-01T18:05:02,2020-01-01 18:15:34,10.0,32,62.50,1
101,2,1,5,2020-01-01T19:00:52,2020-01-01 19:10:54,10.0,27,74.07,1
102,3,1,4,2020-01-02T23:51:23,2020-01-03 00:12:37,21.0,20,67.00,2
102,8,2,3,2020-01-09T23:54:33,2020-01-10 00:15:02,20.0,15,156.00,1
103,4,2,4,2020-01-04T13:23:46,2020-01-04 13:53:03,29.0,40,58.50,3
104,5,3,5,2020-01-08T21:00:29,2020-01-08 21:10:57,10.0,15,66.67,1
104,10,1,4,2020-01-11T18:34:49,2020-01-11 18:50:20,15.0,10,100.00,2
105,7,2,5,2020-01-08T21:20:29,2020-01-08 21:30:45,10.0,25,100.00,1


**<span style="font-size: 14px;">5. If a Meat Lovers pizza was $12 and Vegetarian $10 fixed prices with no cost for extras and each runner is paid $0.30 per kilometre traveled</span>\- how much money does Pizza Runner have left over after these deliveries?**

In [8]:
with cte2 as(
with cte1 as(
SELECT 
	c.order_id,
	c.pizza_id,
	r.runner_id,
	r.distance,
	pn.pizza_name,
	CASE
		WHEN pn.pizza_name='Meatlovers' Then 12 ELSE 10
	END pizza_cost,
	(distance*0.30) as delivery_charge
FROM new_customer_orders c
JOIN new_runner_orders r
	ON c.order_id = r.order_id
JOIN pizza_names pn
	ON c.pizza_id = pn.pizza_id
	WHERE r.cancellation is null)
	SELECT
		order_id,
		delivery_charge,
		SUM(pizza_cost)as overall_pizza_cost_for_order
	FROM cte1
	GROUP BY order_id,delivery_charge
	ORDER BY order_id)
	SELECT
		SUM(overall_pizza_cost_for_order) as total_revenue,
		SUM(delivery_charge) as total_delivery_charges_paid_to_runners,
		SUM(overall_pizza_cost_for_order)-SUM(delivery_charge) as left_over_money
	FROM cte2

(1 row(s) affected)

Total execution time: 00:00:00.002

total_revenue,total_delivery_charges_paid_to_runners,left_over_money
138,43.560,94.440
